In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import TreebankWordTokenizer

import matplotlib
import sklearn
from IPython.core.display import display, HTML

######################################################################################################################

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

get_ipython().run_line_magic('matplotlib', 'inline')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.9)

######################################################################################################################

# TO WORK WITH
import pandas as pd
import numpy as np
from numpy import set_printoptions

# HIDE WARNINGS
import warnings
warnings.filterwarnings('ignore')

# PREPROCESSING & MODEL SELECTION
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV, LassoCV, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import SCORERS
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# print(SCORERS.keys())

# PLOTTING
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")
get_ipython().run_line_magic('matplotlib', 'inline')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn import tree
from graphviz import Source
from matplotlib.pylab import rcParams
import matplotlib.lines as mlines
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import InterclusterDistance
import plotly.express as px
import scipy.cluster.hierarchy as sch
from sklearn.metrics import classification_report


# STANDARD MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# ENSEMBLE
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# XGBOOST
from xgboost import XGBClassifier
import xgboost as xgb

# CLUSTERING
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# PICKLE
from pickle import dump
from pickle import load

from sklearn.metrics import mean_squared_error




In [2]:
    data = pd.read_csv("listings_8.csv")
    data=data.dropna()
    
    data=data[data["price_per_night"]>=12]
    data=data[data["number_of_reviews"]>=1]
    data=data[data["guests_included"]>=1]
    
    

    data['extra_price']=data['security_deposit']+data['cleaning_fee']+data['extra_people']
    data['extra_price'].describe()
    
    
    for test in data['extra_price']:
       
        if test<25:
            data['extra_price']=25
    
    
        if 25<test and test<100:
            data['extra_price']=75
    
    
        if 100<test and test<235:
            data['extra_price']=125
    
    
        if 235<test:
            data['extra_price']=235
            
#             "bedrooms","accommodates", "bathrooms","beds"
    
#     scaler=MinMaxScaler(feature_range=(0,1))
#     data[['extra_price']] = scaler.fit_transform(data[['extra_price']])
#     data.info()
    
    for categorical_feature in ['neighbourhood_cleansed',"property_type","room_type"]:
        data = pd.concat([data,pd.get_dummies(data[categorical_feature], prefix=categorical_feature, prefix_sep='_',)], axis=1)
        
#     data.head()
    
    data= data.drop(['neighbourhood_cleansed', 'property_type', 'room_type','description','host_about_bool',
                    'amenities',"house_rules","review_scores_cleanliness","review_scores_rating",'review_scores_accuracy',              
                    'review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_value',
                     'id',"property_type", "host_response_rate", 'reviews_per_month','number_of_reviews'
                     ,'security_deposit','cleaning_fee','extra_people','extra_price','availability_365'
#                      ,"beds","bathrooms",,"bedrooms","accommodates","guests_included",
                     ],axis=1)
    
    data.info()
    
    y = data["price_per_night"] # Target variable (price_per_night)
    
    X = data.drop(["price_per_night"],axis=1) 
#     print("this is X")
#     X.info()
#     pd.DataFrame(data)
#     scaler=MinMaxScaler(feature_range=(0,1))




count    2181.000000
mean      170.812471
std       192.218113
min         0.000000
25%        25.000000
50%       100.000000
75%       235.000000
max      1024.000000
Name: extra_price, dtype: float64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2181 entries, 0 to 2767
Data columns (total 81 columns):
transit_bool                                      2181 non-null bool
nopets_bool                                       2181 non-null bool
nosmok_bool                                       2181 non-null bool
fewdays_response_time                             2181 non-null bool
1day_response_time                                2181 non-null bool
1hour_response_time                               2181 non-null bool
fewhours_response_time                            2181 non-null bool
host_is_superhost                                 2181 non-null bool
host_identity_verified                            2181 non-null bool
is_location_exact                                 2181 non-null bool
accommodates                                      2181 non-null int64
bathrooms                                         2181 non-null float64
bedrooms                                          2181 non-nu

In [3]:
calendar =pd.read_csv("calendar_cleaned.csv")
calendar.head()
calendar.info()
calendar.describe()
calendar=calendar.drop("available",axis=1)
calendar.info()

,listing_id,date,available,price1
0,3353,2016-12-31,True,32
1,3353,2017-01-01,True,32
2,3353,2017-01-02,True,32
3,3353,2017-01-03,True,32
4,3353,2017-01-04,True,32


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643037 entries, 0 to 643036
Data columns (total 4 columns):
listing_id    643037 non-null int64
date          643037 non-null object
available     643037 non-null bool
price1        643037 non-null int64
dtypes: bool(1), int64(2), object(1)
memory usage: 15.3+ MB


,listing_id,price1
count,6.430370e+05,643037.000000
mean,7.699288e+06,191.691385
std,4.766178e+06,140.786595
min,3.353000e+03,1.000000
25%,3.616081e+06,85.000000
50%,7.789055e+06,150.000000
75%,1.228796e+07,250.000000
max,1.493346e+07,999.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643037 entries, 0 to 643036
Data columns (total 3 columns):
listing_id    643037 non-null int64
date          643037 non-null object
price1        643037 non-null int64
dtypes: int64(2), object(1)
memory usage: 14.7+ MB


In [4]:
# from sklearn.tree import DecisionTreeRegressor

# kfold=KFold(n_splits=15, random_state=7)

# model=DecisionTreeRegressor()
# scoring = "neg_mean_squared_error"

# results=cross_val_score(model, X, y, cv=kfold, scoring=scoring)

# print(f'Decision Trees Regression - MSE {results.mean():.3f} std {results.std():.3f}')


In [5]:
# # KNN Regression 

# from sklearn.neighbors import KNeighborsRegressor

# kfold=KFold(n_splits=10, random_state=7)

# model=KNeighborsRegressor()
# scoring = "neg_mean_squared_error"

# results=cross_val_score(model, X, y, cv=kfold, scoring=scoring)

# print(f'KNN Regression - MSE {results.mean():.3f} std {results.std():.3f}')

In [6]:
# # Linear Regression with scikit-learn

# from sklearn.linear_model import LinearRegression       # if large--> r square
#                                                         # if part of data--> does not work well!! 
# kfold=KFold(n_splits=10, random_state=7)                # sklearn better higher w fi chi bel ma2loub!! 
#                                     #same results BUT DIFFERENT WAY TO USE IT!!
# model=LinearRegression()
# scoring = "neg_mean_squared_error"

# results=cross_val_score(model, X, y, cv=kfold, scoring=scoring)

# print(f'Linear Regression - MSE {results.mean():.3f} std {results.std():.3f}')


# # Now lets use it in the same way than the statsmodel

# model_x=LinearRegression()
# model_x.fit(X,y)
# print(f'Intercept {model_x.intercept_:.4f}')
# print("Coefficients ",model_x.coef_)

# y_pred_x=model_x.predict(X)

# print(f'MAE - Mean Absolute Error {metrics.mean_absolute_error(y, y_pred_x):.3f}')
# print(f'MSE - Mean Square Error  {metrics.mean_squared_error(y, y_pred_x):.3f}')
# print(f'R2    {metrics.r2_score(y, y_pred_x):.3f}')

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 7, alpha = 20, n_estimators = 70)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


[20:22:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=20, base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.2, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=70, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=20, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

RMSE: 62.013981


In [8]:
# 3424.025091074439
# 58.51516975173565
# -1.2280514018653326e+25


# host_is_superhost
# host_identity_verified 
# accommodates                        
# bathrooms                           
# bedrooms                            
# beds                                
# Wheelchair_bool                     
# TV_bool                             
# Hair Dryer_bool                     
# 24-Hour_Check-in_bool               
# Doorman_bool                        
# Gym_bool                            
# Kitchen_bool                        
# Smoke_Detector_bool                 
# Clothes_Dryer_bool                  
# Kitchen_boolean     
# Elevator_in_building_bool 
# Heating_bool

# AC_bool                             
# Internet_bool 

# cleaning_fee                        
# guests_included 

# instant_bookable                    
# super_strict_canc

# require_guest_profile_picture  
# require_guest_phone_verification 

# Allston                             
# Back Bay                            
# Bay Village                     
# Beacon Hill                         
# Brighton                            
# Charlestown                         
# Chinatown                           
# Dorchester                          
# Downtown                            
# East Boston                         
# Fenway                              
# Hyde Park                           
# Jamaica Plain                       
# Leather District                    
# Longwood Medical Area               
# Mattapan                            
# Mission Hill                        
# North End                           
# Roslindale                          
# Roxbury                     
# South Boston                        
# South Boston Waterfront             
# South End                           
# West End                            
# West Roxbury                       
# Entire home/apt                     
# Private room                        
# Shared room   

In [9]:
    
#     data[['security_deposit']] = scaler.fit_transform(data[['security_deposit']])
#     data[['cleaning_fee']] = scaler.fit_transform(data[['cleaning_fee']])
#     data[['host_response_rate']] = scaler.fit_transform(data[['host_response_rate']])
#     data[['price_per_night']] = scaler.fit_transform(data[['price_per_night']])
    
#     data[['accommodates']] = scaler.fit_transform(data[['accommodates']])
#     data[['bathrooms']] = scaler.fit_transform(data[['bathrooms']])
#     data[['bedrooms']] = scaler.fit_transform(data[['bedrooms']])
#     data[['beds']] = scaler.fit_transform(data[['beds']])
#     data[['guests_included']] = scaler.fit_transform(data[['guests_included']])
#     data[['extra_people']] = scaler.fit_transform(data[['extra_people']])
#     data[['availability_365']] = scaler.fit_transform(data[['availability_365']])
#     data[['number_of_reviews']] = scaler.fit_transform(data[['number_of_reviews']])

    
#     data.info()
#     data["host_response_rate"]
#     data.describe()
    
    
    
#     calendar = pd.read_csv("calendar_cleaned.csv")
#     calendar.describe()
#     calendar.head()
    
    
    
#     data=data[["host_is_superhost", "host_identity_verified", "accommodates", "bathrooms", "bedrooms",
#          "beds", "Wheelchair_bool", "TV_bool", "Hair Dryer_bool", "24-Hour_Check-in_bool", "Doorman_bool",
#          "Gym_bool", "Kitchen_bool", "Smoke_Detector_bool", "Clothes_Dryer_bool", "Kitchen_boolean",
#          "Elevator_in_building_bool", "Heating_bool", "AC_bool", "Internet_bool", "cleaning_fee",
#          "guests_included", "instant_bookable", "super_strict_canc", "require_guest_profile_picture",
#          "require_guest_phone_verification", "Allston", "Back Bay", "Bay Village", "Beacon Hill", "Brighton",
#          "Charlestown", "Chinatown", "Dorchester", "Downtown", "East Boston", "Fenway", "Hyde Park", "Jamaica Plain",
#          "Leather District", "Longwood Medical Area", "Mattapan", "Mission Hill", "North End", "Roslindale", "Roxbury",
#          "South Boston", "South Boston Waterfront", "South End", "West End", "West Roxbury", "Entire home/apt", "Private room",
#          "Shared room","price_per_night","cleaning_fee"]]
    
#     for col in data.columns: 
#         print(col)
#         print(col.types)
       
    
    
#     data.info()

#     print("HI INFO")
#     data.describe()
#     data.info()
    
#     data.describe()

    
    
#     kfold=KFold(n_splits=10, random_state=7)
#     model=LinearRegression()
#     scoring = "neg_mean_squared_error"
    
# #     y_pred = clf.predict(data.drop('price_per_night', axis='columns'))
    
#     results=cross_val_score(model, X_wnei, y, cv=kfold, scoring=scoring)
    
#     clf = model.fit(X_wnei, y)
    
#     MSEs=[]
#     MSEs.append(("Linear Regression", results.mean(), clf.coef_))
#     print(MSEs)
    
    
#     from sklearn.model_selection import KFold # import KFold
    
    
#     kf = KFold(n_splits=2) # Define the split - into 2 folds 
#     kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
#     print(kf) 
#     KFold(n_splits=2, random_state=None, shuffle=False)

#     for train_index, test_index in kf.split(X):
#     print(“TRAIN:”, train_index, “TEST:”, test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     ('TRAIN:', array([2, 3]), 'TEST:', array([0, 1]))
#     ('TRAIN:', array([0, 1]), 'TEST:', array([2, 3]))



# ---------------------------------------------------------------

#THIS IS THE BEST ONE!

#     from sklearn.linear_model import LinearRegression
#     from sklearn.metrics import mean_squared_error
    
#     data_train, data_test, price_train, price_test = train_test_split(X, y,test_size=0.25,random_state=7)
    
#     model=LinearRegression()
    
#     model.fit(data_train,price_train)
    
#     predicted_price=model.predict(data_train)
    
#     mean_squared_error(price_train, predicted_price)
    
#     np.sqrt(mean_squared_error(price_train, predicted_price))  
    
    
#     from sklearn.model_selection import cross_val_score
    
#     ols2 = LinearRegression()
#     ols_cv_mse = cross_val_score(ols2, data_train, price_train, scoring='neg_mean_squared_error', cv=4)
#     ols_cv_mse.mean()

# -------------------------------------------------------------------


# XGBoost
#  evaluated with train & test - remember we have a high variance !  
# it does everything and almost 1% better that's why people love it
# # we need more test data!!!
#     seed=7
#     test_size=0.22

# # split into train and test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)



# #############################################################

# # XGBoost 
# #   evaluated with KFold
# #   in this case we use 3 splits because the amount of data is not large

#     kfold=KFold(n_splits=3, random_state=seed)
    
#     learner=DecisionTreeClassifier(class_weight="balanced", random_state=seed)
#     learner=xgb.XGBRegressor()

#     xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1, learning_rate = 0.01,
#                          max_depth = 6, alpha = 4, n_estimators = 1000)

# # learning_rate: 0.01
# # n_estimators: 100 if the size of your data is high, 1000 is if it is medium-low
# # max_depth: 3
# # subsample: 0.8
# # colsample_bytree: 1
# # gamma: 1

#     xg_reg.fit(X_train,y_train)

#     preds = xg_reg.predict(X_test)

#     rmse = np.sqrt(mean_squared_error(y_test, preds))

#     print("RMSE: %f" % (rmse))


#     eval_set = [(X_train, y_train), (X_test, y_test)]

#     eval_metric = ["auc","error"]

    
#     model.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_test, y_test)],eval_metric=["auc","error"], verbose=True)


#     results=cross_val_score(model, X, y, cv=kfold)

#     print(f'XGBoost with kfold - Accuracy {results.mean()*100:.3f}% std {results.std()*100:3f}')

#######################################################










#     seed=7
#     kfold=KFold(n_splits=10, random_state=seed) #KFold



#     num_trees=100
#     num_features=4



#     X_train,X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=seed) #Seperating into tranin and test



#     model=RandomForestClassifier(n_estimators=num_trees, max_features=num_features, random_state=seed) #model
    
#     results=cross_val_score(model, X, y, cv=kfold) #CV



#     # Accuracy Before Pickling the model
#     print(f'Random Forest - Accuracy {results.mean()*100:.3f}% std {results.std()*100:3f}')



#     model.fit(X_train,y_train)



#     # Now we save it into a file
#     filename="PaulaMoya_RFC_model.sav"
#     dump(model, open(filename, "wb"))
#     # .... some time later .... 
#     #load the model from disk
#     loaded_model=load(open(filename, "rb"))
#     result=loaded_model.score(X_test,y_test)



#     # Accuracy After Pickling and loading the model
#     print(f'Loaded model - Accuracy {results.mean()*100:.3f}% std {results.std()*100:3f}')
    
    
    
    
    
    

    
#     from sklearn.linear_model import LinearRegression       # if large--> r square
#                                                             # if part of data--> does not work well!! 
#     kfold=KFold(n_splits=2, random_state=7)                # sklearn better higher w fi chi bel ma2loub!! 
#                                         #same results BUT DIFFERENT WAY TO USE IT!!
#     model=LinearRegression()
#     scoring = "neg_mean_squared_error"

#     results=cross_val_score(model, X, y, cv=kfold, scoring=scoring)

#     print(f'Linear Regression - MSE {results.mean():.3f} std {results.std():.3f}')

#     # res_w1["Res"]=results
#     # res_w1["Type"]="Lin"

#     # resall=pd.concat([resall,res_w1], ignore_index=True)

#     # Now lets use it in the same way than the statsmodel
    
    
    
#     len(data["host_response_rate"])

#     model_x=LinearRegression()
#     model_x.fit(X,y)
#     print(f'Intercept {model_x.intercept_:.4f}')
#     print("Coefficients ",model_x.coef_)

#     y_pred_x=model_x.predict(X)

#     print(f'MAE - Mean Absolute Error {metrics.mean_absolute_error(y, y_pred_x):.3f}')
#     print(f'MSE - Mean Square Error  {metrics.mean_squared_error(y, y_pred_x):.3f}')
#     print(f'R2    {metrics.r2_score(y, y_pred_x):.3f}')    
    
    
#     from sklearn.linear_model import LinearRegression
    
#     clf = LinearRegression()
#     y = data['price_per_night']
#     clf.fit(data.drop('price_per_night', axis='columns'), y)

#     y_pred = clf.predict(data.drop('price_per_night', axis='columns'))
    
#     import sklearn.metrics
#     mse = sklearn.metrics.mean_squared_error(y, y_pred)
#     print("mse:")
#     mse
    
    
#     root_mse = mse**(1/2)
#     print("root_mse:")
#     root_mse    #by how much we're missing out
    
#     print("r_squared:")
#     r_squared = sklearn.metrics.r2_score(y, y_pred)
#     r_squared          #how much of the variance are we predicting
    
    # WE STILL HAVE ID TO LINK IT TO THE CALENDAR TABLE!

#     y = data["price_per_night"] # Target variable (price_per_night)
#     X_wnei = data.drop(["price_per_night"],axis=1)
    
#     # Creating new DF without neighborhood names
#     X_wnei.to_csv("X_wnei.csv", index=False)
#     data.to_csv("data.csv", index=False)